In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/CS7643/Project'

/content/drive/MyDrive/CS7643/Project


In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers

In [5]:
from transformers import BertTokenizer, BertModel

#tokenizer = BertTokenizer.from_pretrained('data/tokenizer/fake_tokenizer', max_len=512)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from datasets import load_dataset

In [7]:
# All data will be loaded together unless data_files is specified
dataset = load_dataset('data', data_files ={'train':'train.csv','test':'test.csv'})
dataset = dataset.rename_column("Unnamed: 0", 'id')

Using custom data configuration data-cf9ebd5eebbe0a50
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/data-cf9ebd5eebbe0a50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
dataset = dataset.map(lambda e: tokenizer(e['text'], add_special_tokens = True, max_length = 64, truncation = True, padding = 'max_length'))

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/data-cf9ebd5eebbe0a50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-51ef8476bc4af83e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/data-cf9ebd5eebbe0a50/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-aaa944c4ffb770de.arrow


In [9]:
dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [10]:
dataset.column_names

{'test': ['id',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'train': ['id',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

In [11]:
test  = dataset['test']

In [12]:
test.column_names

['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

In [13]:
test.shape

(8980, 6)

In [14]:
texts = test['text']

In [15]:
len(texts)

8980

In [16]:
tokens = test['input_ids']

In [17]:
texts[0]

'21st Century Wire says Ben Stein, reputable professor from, Pepperdine University (also of some Hollywood fame appearing in TV shows and films such as Ferris Bueller s Day Off) made some provocative statements on Judge Jeanine Pirro s show recently. While discussing the halt that was imposed on President Trump s Executive Order on travel. Stein referred to the judgement by the 9th Circuit Court in Washington state as a  Coup d tat against the executive branch and against the constitution.  Stein went on to call the Judges in Seattle  political puppets  and the judiciary  political pawns. Watch the interview below for the complete statements and note the stark contrast to the rhetoric of the leftist media and pundits who neglect to note that no court has ever blocked any Presidential orders in immigration in the past or discuss the legal efficacy of the halt or the actual text of the Executive Order.READ MORE TRUMP NEWS AT: 21st Century Wire Trump FilesSUPPORT OUR WORK BY SUBSCRIBING &

In [18]:
tokens

tensor([[  101,  7398,  2301,  ...,  1055,  3237,   102],
        [  101,  2899,  1006,  ...,  2011,  1037,   102],
        [  101,  1006, 26665,  ...,  5057, 23701,   102],
        ...,
        [  101,  7398,  2301,  ...,  3373,  1010,   102],
        [  101,  1997,  2607,  ...,  2001,  8112,   102],
        [  101, 16371,  9681,  ...,     0,     0,     0]])

In [19]:
tokens.dtype

torch.int64

In [20]:
tokenizer.decode(tokens[0])

'[CLS] 21st century wire says ben stein, reputable professor from, pepperdine university ( also of some hollywood fame appearing in tv shows and films such as ferris bueller s day off ) made some provocative statements on judge jeanine pirro s show recently. while discussing the halt that was imposed on president trump s executive [SEP]'

In [21]:
dataset.column_names

{'test': ['id',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask'],
 'train': ['id',
  'text',
  'label',
  'input_ids',
  'token_type_ids',
  'attention_mask']}

In [26]:
shards = test.shard(num_shards = 12, index = 0)

In [27]:
test.shape

(8980, 6)

In [28]:
shards.shape

(749, 6)

In [29]:
shards.column_names

['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask']

In [30]:
shards['input_ids'].shape

torch.Size([749, 64])

# This worked but was not parallelized and not on the GPU so took forever for just a test size amount of data

In [46]:
# get contextualized embeddings from bert model
import torch
from time import time
start = time()
bert_embeddings = []
num_shards = 12
with torch.no_grad():
    for s in range(1,num_shards - 1):
      shards = test.shard(num_shards = 12, index = s)
      bert_embeddings.append(model(shards['input_ids'])[0])
    # X_train_bert = bert_model(train_indices)[0]  # Models outputs are tuples
    # X_val_bert = bert_model(val_indices)[0]
    # X_test_bert = bert_model(test_indices)[0]
end = time()
elapsed = end - start
if elapsed < 180:
    print(f'code took {elapsed:0.2f} seconds to execute')
else:
    print(f'code took {elapsed / 60:0.2f} minutes to execute')

code took 19.18 minutes to execute


In [47]:
from torch.utils.data import DataLoader, SequentialSampler

dataloader = DataLoader(dataset= test['input_ids'], batch_size = 256, sampler = SequentialSampler)

In [48]:
!pip install tqdm

### TODO: Get the dataloader working with the test['input_ids']

REF:  https://discuss.huggingface.co/t/how-to-ensure-fast-inference-on-both-cpu-and-gpu-with-bertforsequenceclassification/1694

In [49]:
#Ref: https://discuss.huggingface.co/t/how-to-ensure-fast-inference-on-both-cpu-and-gpu-with-bertforsequenceclassification/1694
from tqdm import tqdm_notebook
import numpy as np

# Get the progress bar for later modification
progress_bar = tqdm_notebook(dataloader, ascii=True)

for batch_idx, data in enumerate(progress_bar):
  with torch.no_grad():        
    outputs = model(data)

    logits = outputs[0]

    all_logits = np.vstack([all_logits, torch.softmax(logits, dim=1).detach().cpu().numpy()])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


0it [00:00, ?it/s]

TypeError: ignored